In [1]:
import os, time, shutil, tarfile
from minio import Minio
from minio.error import ResponseError
from tqdm import tnrange

times = 10
resultSmallFile = []
resultBigFile = []
testList = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

Gen test file

In [2]:
with open('file.txt', 'w') as f:
    for i in range(64*1000*1000):
        f.write('a')

os.mkdir('minio_test_data')

for size in testList:
    os.mkdir('minio_test_data/'+str(size))
    tar=tarfile.open('minio_test_data/'+str(size)+'.tar','w')  
    for i in range(size):
        shutil.copyfile('file.txt', 'minio_test_data/'+str(size)+'/'+str(i))
        tar.add('minio_test_data/'+str(size)+'/'+str(i))
    tar.close()

Upload test

In [ ]:
minioClient = Minio('140.124.181.78:30158',
                    access_key='wei840222',
                    secret_key='j04rmp4jo3',
                    secure=False)

for ti in tnrange(times, desc='times'):
    runTimeListSmallFile = []
    runTimeListBigFile = []
    
    for size in testList:
        try:
            minioClient.make_bucket('test-size'+str(size)+'-'+str(ti), location="us-east-1")
        except ResponseError as err:
            print(err)

        try:
            start = time.time()
            for j in tnrange(size, desc='files'):
                minioClient.fput_object('test-size'+str(size)+'-'+str(ti), str(j), 'minio_test_data/'+str(size)+'/'+str(j))
            end = time.time()
            runTimeListSmallFile.append(end-start)
        except ResponseError as err:
            print(err)
            print('upload minio_test_data/'+str(size)+'/'+str(j)+'fail')
            runTimeListSmallFile.append(-1)
            
        try:
            start = time.time()
            minioClient.fput_object('test-size'+str(size)+'-'+str(ti), str(size)+'.tar', 'minio_test_data/'+str(size)+'.tar')
            end = time.time()
            runTimeListBigFile.append(end-start)
        except ResponseError as err:
            print(err)
            print('upload minio_test_data/'+str(size)+'.tar fail')
            runTimeListBigFile.append(-1)
            
    resultSmallFile.append(runTimeListSmallFile)
    resultBigFile.append(runTimeListBigFile)

output result to csv

In [ ]:
with open('minio/small_file_nfs_upload.csv', 'w') as f:
    f.write(str([str(t)+'-file' for t in testList]).replace("'",'')[1:-1]+'\n')
    for r in resultSmallFile:
        f.write(str(r)[1:-1]+'\n')
with open('minio/big_file_nfs_upload.csv', 'w') as f:
    f.write(str([str(t)+'-file' for t in testList]).replace("'",'')[1:-1]+'\n')
    for r in resultBigFile:
        f.write(str(r)[1:-1]+'\n')

Download test

In [ ]:
resultSmallFile = []
resultBigFile = []
for ti in tnrange(times, desc='times'):
    runTimeListSmallFile = []
    runTimeListBigFile = []
    
    for size in testList:
        try:
            start = time.time()
            for j in tnrange(size, desc='files'):
                minioClient.fget_object('test-size'+str(size)+'-'+str(ti), str(j), 'minio_test_data_read/'+str(size)+'/'+str(j))
            end = time.time()
            runTimeListSmallFile.append(end-start)
        except ResponseError as err:
            print(err)
            print('download minio_test_data/'+str(size)+'/'+str(j)+'fail')
            runTimeListSmallFile.append(-1)
            
        try:
            start = time.time()
            minioClient.fget_object('test-size'+str(size)+'-'+str(ti), str(size)+'.tar', 'minio_test_data_read/'+str(size)+'.tar')
            end = time.time()
            runTimeListBigFile.append(end-start)
        except ResponseError as err:
            print(err)
            print('download minio_test_data/'+str(size)+'.tar fail')
            runTimeListBigFile.append(-1)
            
    resultSmallFile.append(runTimeListSmallFile)
    resultBigFile.append(runTimeListBigFile)

In [ ]:
with open('minio/small_file_nfs_download.csv', 'w') as f:
    f.write(str([str(t)+'-file' for t in testList]).replace("'",'')[1:-1]+'\n')
    for r in resultSmallFile:
        f.write(str(r)[1:-1]+'\n')
with open('minio/big_file_nfs_download.csv', 'w') as f:
    f.write(str([str(t)+'-file' for t in testList]).replace("'",'')[1:-1]+'\n')
    for r in resultBigFile:
        f.write(str(r)[1:-1]+'\n')

clean file

In [ ]:
try:
    os.remove('file.txt')
except OSError as e:  ## if failed, report it back to the user ##
    print ("Error: %s - %s." % (e.filename, e.strerror))
try:
    shutil.rmtree('minio_test_data')
except OSError as e:
    print ("Error: %s - %s." % (e.filename, e.strerror))
try:
    shutil.rmtree('minio_test_data_read')
except OSError as e:
    print ("Error: %s - %s." % (e.filename, e.strerror))